### Importation des données

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import yaml
import importlib

from utils import HTMLParser

In [3]:
data = pd.read_csv("data/train.csv")

In [4]:
# Génération du dictionnaire .json à partir du HTML
parser = HTMLParser()
html_snippet = "data/USCODE22_LLCP_102523.HTML"

codebook = parser.parse_html_codebook(html_snippet)

In [8]:
config = "features.yaml"

with open(config, 'r') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [9]:
data_reduced = data[config['features'] + ['TARGET'] + ["ID"]].copy()

In [ ]:
features_classification = {}

for feature in config['features']:
    features_classification[feature] = {}
    entry = next((item for item in codebook if item["sas_variable"] == feature), None)
    if entry:
        categories = list(set([cat['value'] for cat in entry['categories']]))
        categories.sort()
        if categories[0] != str(1):
            features_classification[feature]['type'] = 'numerical'

            print(f"Feature: {feature}")
            print(categories)

        else:
            features_classification[feature]['type'] = 'categorical'
    else:
        features_classification[feature]['type'] = 'unknown'

Feature: PHYSHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: MENTHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: POORHLTH
['1 - 30', '77', '88', '99', 'BLANK']
Feature: SLEPTIM1
['1 - 24', '77', '99', 'BLANK']
Feature: WEIGHT2
['50 - 0776', '7777', '9023 - 9352', '9999', 'BLANK']
Feature: HEIGHT3
['200 - 711', '7777', '9061 - 9998', '9999', 'BLANK']
Feature: MARIJAN1
['1 - 30', '77', '88', '99', 'BLANK']
Feature: ALCDAY4
['101 - 199', '201 - 299', '777', '888', '999', 'BLANK']
Feature: AVEDRNK3
['1 - 76', '77', '88', '99', 'BLANK']
Feature: DRNK3GE5
['1 - 76', '77', '88', '99', 'BLANK']
Feature: MAXDRNKS
['0', '1 - 76', '77', '99', 'BLANK']
Feature: COPDSMOK
['1 - 76', '77', '88', '99', 'BLANK']
Feature: _PACKDAY
['0 - 100', 'BLANK']


In [26]:
# Handle missing values for numerical features